In [ ]:
repo = "FairMOT"
%cd "/kaggle/working"
!rm -rf {repo}
!git clone https://github.com/ptran1203/{repo}
%cd {repo}
!pip install -r requirements.txt

%cd src/lib/model/networks/
!rm -rf DCNv2
!git clone https://github.com/ifzhang/DCNv2/
%cd DCNv2
!git checkout "pytorch_1.7"
!./make.sh
!pip install gdown
%cd /kaggle/working/{repo}

In [ ]:
import torch
print(torch.__version__)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())
import numpy as np
import seaborn as sns
import sys
import os
import json
import os.path as osp
from glob import glob
import pandas as pd
from sklearn.model_selection import GroupKFold
SRC_DIR = '/kaggle/input/nfl-data/dataset'
IMG_DIR = f'{SRC_DIR}/images'


## Create data config

In [ ]:
## Split data
test_videos = ['58094_000423_Endzone.mp4', '57586_000540_Sideline.mp4', '57586_000540_Endzone.mp4',
        '57584_000336_Endzone.mp4', '57584_000336_Sideline.mp4', '58093_001923_Sideline.mp4', '57911_000147_Sideline.mp4',
        '57778_004244_Sideline.mp4', '57586_004152_Endzone.mp4', '58106_002918_Sideline.mp4']

gkfold = GroupKFold(n_splits=4)
imgs = os.listdir(IMG_DIR)  # not sort yet
img_df = pd.DataFrame(imgs, columns=['image'])
img_df['video'] = img_df.image.apply(lambda x: x.split('.mp4')[0] + '.mp4')
img_df['frame'] = img_df.image.apply(lambda x: x.split('.mp4_')[1].split('.')[0]).astype('int64')
img_df['group'] = img_df.groupby('video').ngroup()

print('Videos', len(img_df['video'].unique()))
img_df = img_df.sort_values(['video', 'frame'])
dum = np.zeros(len(img_df))
for i, (train_index, test_index) in enumerate(gkfold.split(img_df,  img_df['group'], img_df['group'])):
    split = 'val' if i == 0 else 'train'
    img_df.loc[test_index, 'subset'] = split

img_df.loc[img_df.video.isin(test_videos), 'subset'] = 'val'
trains = img_df[img_df.subset == "train"]
vals = img_df[img_df.subset == "val"]
# img_df = img_df.sample(frac=0.1)
print(f'Train on {len(trains)}, valid on {len(vals)}')
img_df.query('subset == "train"').head()

In [ ]:
# Step 1. Generate files containing image paths and save to src/data
train_file = 'src/data/nfl.train'
test_file = 'src/data/nfl.val'

with open(train_file, 'w') as f:
    f.write("\n".join(img_df.query('subset == "train"').image.values))
with open(test_file, 'w') as f:
    f.write("\n".join(img_df.query('subset == "val"').image.values))

# Step 2. Create a json file for the dataset in src/lib/cfg/
cfg = dict(
    root=IMG_DIR,
    train=dict(nfl=train_file),
    test=dict(nfl=test_file),
    test_emb=dict(nfl=test_file),
)
with open('src/lib/cfg/nfl_data.json', 'w') as f:
    json.dump(cfg, f, indent=4)

In [ ]:
# Download pre-trained weight to continue training
!gdown https://drive.google.com/u/0/uc?id=1ujB57OSV4r0SxwgDwrwylNHvbceTxRsW

In [ ]:
!python src/train.py mot --data_cfg "src/lib/cfg/nfl_data.json" --gpus 0\
                        --arch resdcn_34\
                        --load_model /kaggle/working/FairMOT/model_last.pth\
                        --resume

In [ ]:
# Zip weight and log to download
!rm /kaggle/working/fairmot_exp_resdcn34.zip 
!zip /kaggle/working/fairmot_exp_resdcn34.zip exp/mot/default/model_last.pth exp/mot/default/logs_*/log.txt
!du -sh /kaggle/working/fairmot_exp_resdcn34.zip 

In [ ]:
 !python src/demo.py mot --load_model /kaggle/working/FairMOT/model_last.pth\
                        --input-video "/kaggle/input/nfl-health-and-safety-helmet-assignment/train/57584_000336_Endzone.mp4"

In [ ]:
!zip -r /kaggle/working/demos.zip /kaggle/working/demos/*